# TensorFlow ディープニューラルネットワーク

#### MNIST と TensorFlow を使ったディープラーニングチュートリアル
by [@kakauandme](https://twitter.com/KaKaUandME) and [@thekoshkina](https://twitter.com/thekoshkina)

精度: 0.99

**前提条件:** 基礎的なコーディング能力、線形代数（特に行列計算）。画像がコンピュータのメモリ上、どのように保存されているかを知っているとよりよく理解できるしょう。機械学習を始めるには、Andrew Ng 氏による次のサイトをオススメします。[coursera course](https://www.coursera.org/learn/machine-learning)


備考:

*自由に[このカーネル](https://www.kaggle.com/kakauandme/tensorflow-deep-nn)をフォークして CONSTANTS を調整し、ネットワークの動きがどう変わるか、アルゴリズムのパフォーマンスや精度がどれくらい変わるかをチェックしてみてください。さらに **TensorFlow グラフ** 節もまた、学習のためには変更してみるとよいでしょう。*

*100% 理解できないようであれば、変数を都度出力することをオススメします。また、ローカル環境であれば [tensorboard](https://www.tensorflow.org/versions/master/how_tos/summaries_and_tensorboard/index.html) を可視化やデバッグのために利用することもできます。*

## ライブラリの導入と設定

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf

# 設定
LEARNING_RATE = 1e-4

# ローカル環境で 0.99 の精度を得たければ、20000 をセットしましょう
TRAINING_ITERATIONS = 2500        
    
DROPOUT = 0.5
BATCH_SIZE = 50

# 全てを学習データとしてのみ扱うなら 0 をセットします
VALIDATION_SIZE = 2000

# 試しに出力してみる画像のインデックス
IMAGE_TO_DISPLAY = 10

## データの準備
機械学習を始める前に、まずデータを読んでみましょう。*train.csv* は 42,000 行、785 カラムの CSV ファイルです。各行は「手書き数字」と「実際のその値」を表現しています。

In [ ]:
# CSV ファイルから学習データを読み込みます
data = pd.read_csv('./train.csv')

print('data({0[0]}, {0[1]})'.format(data.shape))
print(data.head())

各画像は、ピクセルの配列に "引き伸ばされた" 状態です。

In [ ]:
images = data.iloc[:,1:].values
images = images.astype(np.float)

# 後続処理のために値を変換します [0:255] => [0.0:1.0]
images = np.multiply(images, 1.0 / 255.0)

print('images({0[0]}, {0[1]})'.format(images.shape))

今回は 784 ピクセルで正方形、つまり 28 * 28px の画像です。

In [ ]:
image_size = images.shape[1]
print('image_size => {0}'.format(image_size))

# 今回すべての画像は正方形です
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)

print('image_width => {0}\nimage_height => {1}'.format(image_width,image_height))

画像を出力するには、この長いピクセル文字列を基本的に白黒画像である 2次元配列に加工します。

In [ ]:
# 画像の表示
def display(img):
    
    # (784) => (28, 28)
    one_image = img.reshape(image_width, image_height)
    
    plt.axis('off')
    plt.imshow(one_image, cmap=cm.binary)

# 出力    
display(images[IMAGE_TO_DISPLAY])

結果として取りうるラベルは 0 から 9 までの数字であり、それは画像そのものが指す数字となります。

In [ ]:
labels_flat = data.iloc[:,0].values

print('labels_flat({0})'.format(len(labels_flat)))
print ('labels_flat[{0}] => {1}'.format(IMAGE_TO_DISPLAY, labels_flat[IMAGE_TO_DISPLAY]))

この学習データには 10 種類の 数字 / ラベル / クラス があることを確認しましょう。

In [ ]:
labels_count = np.unique(labels_flat).shape[0]

print('labels_count => {0}'.format(labels_count))

ほとんどのクラス分類問題では "One-hot" ベクトルが使われます。"One-hot" ベクトルとは、ひとつの要素のみが 1 で、そのほかの要素がすべて 0 であるベクトルです。今回、数字 *n* は *n 番目* が 1 である配列、として表現することにしましょう。

In [ ]:
# クラスのラベルをスカラ値から one-hot ベクトルへ変換
# 0 => [1 0 0 0 0 0 0 0 0 0]
# 1 => [0 1 0 0 0 0 0 0 0 0]
# ...
# 9 => [0 0 0 0 0 0 0 0 0 1]
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

labels = dense_to_one_hot(labels_flat, labels_count)
labels = labels.astype(np.uint8)

print('labels({0[0]}, {0[1]})'.format(labels.shape))
print('labels[{0}] => {1}'.format(IMAGE_TO_DISPLAY,labels[IMAGE_TO_DISPLAY]))

最後に、入力データを一部検証用に分割しておきましょう。これは機械学習では必須です。学習には用いられないデータセットを別途用意することで、学習したものが実際うまく一般化されたかを確認することに使えます。

In [ ]:
# 学習 & 検証用に、データを分割する
validation_images = images[:VALIDATION_SIZE]
validation_labels = labels[:VALIDATION_SIZE]

train_images = images[VALIDATION_SIZE:]
train_labels = labels[VALIDATION_SIZE:]

print('train_images({0[0]}, {0[1]})'.format(train_images.shape))
print('validation_images({0[0]}, {0[1]})'.format(validation_images.shape))

*データの準備はできました。次はニューラルネットワークの構造を定義していきましょう。*

## TensorFlow グラフ

TensorFlow は Python でないところで重い処理を実行します。そのためすべての操作を独立して実行する代わりに、対話的な操作をひとつの「グラフ」として利用者に定義させ、すべての処理は分離された別プロセスで実行される仕組みとなっています。

#### ヘルパー関数

ニューラルネットワークモデルのために、たくさんの Weight（重み付け）と Bias が生成されることになります。一般的にいって、Weight は "対称性の破れ" を起こし "ゼロ勾配" とならないよう、微量のノイズと共に初期化されるべきです。

 [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks) ニューロン（ *f(x)=max(0,x)* という関数を含むもの）を使うので、"ニューロンの死" を避けるために少しだけ正値のバイアスを初期化に用いいることもよい方法です。

In [ ]:
# Weight 初期化
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

今回はゼロ埋めした [畳み込み](https://en.wikipedia.org/wiki/Convolutional_neural_network#Convolutional_layer) を使い、出力サイズを入力サイズと一致させます。この例ではストライド（CNN 用語）をステップで割った値は 1 となります。

一般に、畳み込みレイヤはデータの特徴を抽出することに使われます。形から数字を読む数字認識においていえば、それぞれ特定の形に反応するカーネルやフィルタ（いずれも CNN 用語）を使うことになるでしょう。フィルタの数は解きたい問題によって変わり得ます。

In [ ]:
# 畳み込み
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

[プーリング](https://en.wikipedia.org/wiki/Convolutional_neural_network#Pooling_layer) は 2x2 ブロックの純粋な Max Pooling 関数です。

プーリングは重要な情報は残しながら、扱うデータを小さくすることに使われます。2x2 の Max Pooling 関数は画像を 2 ピクセル四方のブロックに分割し、そのブロック内での最大値のみを返します。

In [ ]:
# プーリング
# [[0,3],
#  [4,2]] => 4

# [[0,1],
#  [1,1]] => 1

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

*畳み込みとプーリングについては後でまた詳細に踏み込みます*

どんなニューラルネットワークでもとても深いレイヤに組み込めるというメリットが意味するのは、ひとつのレイヤの結果が、次のレイヤの入力として利用できるということです。この逐次的アプローチを使えば、とても繊細な、とても深いニューラルネットのレイヤが構成できます。これをディープニューラルネットワークと呼びます。

今回はプーリング層を間に挟んだ 2 つの畳み込み層に続け、全結合層、ドロップアウト、そして出力層で構成します。

In [ ]:
# ニューラルネットワークの入力 & 出力

# 画像
x = tf.placeholder('float', shape=[None, image_size])
# ラベル
y_ = tf.placeholder('float', shape=[None, labels_count])

最初の層は畳み込み、次に Max pooling と続きます。畳み込みでは各 5x5 ピクセルごと、32 の特徴が計算されます。この時 Weight テンソルは [5, 5, 1, 32] という形となります。最初の 2 次元はパッチサイズ、3 つ目は入力チャンネル（最初の層では色味、つまり 1 とは白黒のこと）、最後のものは出力サイズです。バイアス配列もまた、各出力チャンネルに配置されています。

このレイヤを適用するために、入力データも 4 次元のテンソルに整形します。最初の次元は画像数、2、3 番目は画像の幅と高さ、4 番目は色チェンネルです。

畳み込みのあとは、プーリング層によって出力サイズが 28x28 から 14x14 に縮小されます。

In [ ]:
# 1 つ目の畳み込み層
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# (40000,784) => (40000,28,28,1)
image = tf.reshape(x, [-1,image_width , image_height,1])
# print(image.get_shape()) # =>(40000,28,28,1)

h_conv1 = tf.nn.relu(conv2d(image, W_conv1) + b_conv1)
# print(h_conv1.get_shape()) # => (40000, 28, 28, 32)
h_pool1 = max_pool_2x2(h_conv1)
# print(h_pool1.get_shape()) # => (40000, 14, 14, 32)

# 可視化のための準備
# 4x8 の行列として 32 の特徴を表示
layer1 = tf.reshape(h_conv1, (-1, image_height, image_width, 4 ,8))  

# チャンネルが最初の次元になり、それに x, y が続く様に並び替え
layer1 = tf.transpose(layer1, (0, 3, 1, 4,2))
layer1 = tf.reshape(layer1, (-1, image_height*4, image_width*8)) 

2 番目のレイヤは 5x5 ピクセルごとに、64 の特徴を抽出します。Weight テンソルは [5, 5, 32, 64] という形をしています。最初の 2 次元はパッチサイズ、その次は入力チャンネル数（この前のレイヤで得たのは 32 の特徴でしたね）、そして最後は出力チャンネル数です。バイアス配列もまた、各出力チャンネルに配置されています。

画像はプーリングによって 14x14 まで縮小されているため、2 つ目の畳み込み層では、画像のより一般的な特徴をピックアップします。フィルタの適用範囲はより広くなります。それにより、最初の層ではディテールをみていましたが、2 層目ではより総括的な特徴抽出が可能になります。

In [ ]:
# 2 つ目の畳み込み層
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
# print(h_conv2.get_shape()) # => (40000, 14,14, 64)
h_pool2 = max_pool_2x2(h_conv2)
# print(h_pool2.get_shape()) # => (40000, 7, 7, 64)

# 可視化のための準備
# 4x16 の行列として 64 の特徴を表示
layer2 = tf.reshape(h_conv2, (-1, 14, 14, 4 ,16))  

# チャンネルが最初の次元になり、それに x, y が続く様に並び替え
layer2 = tf.transpose(layer2, (0, 3, 1, 4,2))
layer2 = tf.reshape(layer2, (-1, 14*4, 14*16)) 

画像サイズが 7x7 まで縮小されたので、ここで 1024 個のニューロンからなる [全結合層](https://en.wikipedia.org/wiki/Convolutional_neural_network#Fully_Connected_layer) を使い、いよいよ画像全体での処理結果を求めてみましょう。（全結合層の各ニューロンは、前の層のすべての活性化関数 / 出力と繋がっています）

In [ ]:
# 全結合
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# (40000, 7, 7, 64) => (40000, 3136)
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# print(h_fc1.get_shape()) # => (40000, 1024)

過学習を防止するために、出力層の前に[ドロップアウト](https://en.wikipedia.org/wiki/Convolutional_neural_network#Dropout) を使います。

ドロップアウトは各学習段階のネットワークから、いくつかのノードを除外します。各ノードの値は確率 *keep_prob* に応じて伝播されるか、確率 *1 - keep_prob* に応じて省かれます。学習段階が終わった後は、各ノードは初期の Weight でニューラルネットワークに戻されます。

In [ ]:
# ドロップアウト
keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

最後に、Softmax 層を追加します。単純な [Softmax 回帰](https://en.wikipedia.org/wiki/Softmax_function) と同じです。

In [ ]:
# 深層のための出力層
W_fc2 = weight_variable([1024, labels_count])
b_fc2 = bias_variable([labels_count])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# print(y.get_shape()) # => (40000, 10)

ネットワークのパフォーマンスを評価するために、[クロスエントロピー](https://en.wikipedia.org/wiki/Cross_entropy) を使い、それを最小化するために [ADAM 最適化](http://arxiv.org/pdf/1412.6980v8.pdf) を使います。

ADAM 最適化は勾配ベースの最適化アルゴリズムであり、適応性のある推定に基づいています。急激な勾配降下法よりも洗練されていて、巨大なデータや膨大なパラメタとなる課題に適しています。

In [ ]:
# コスト関数
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# 最適化関数
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

# 評価
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

テストデータから推論するには、0 から 9 の数字のうちもっとも高い可能性だと示唆する "One-hot ベクトル" から、もっとも高い確率のものが選択されます。

In [ ]:
# 推論関数
#[0.1, 0.9, 0.2, 0.1, 0.1 0.3, 0.5, 0.1, 0.2, 0.3] => 1
predict = tf.argmax(y, 1)

*最後にニューラルネットワークの構造が決まったら、学習に向けての TensorFlow グラフの準備は万端です。*

## 学習、検証そして推論

#### ヘルパー関数

理想的には学習の各ステップですべてのデータを使うべきですが、それでは高くつきます。そこで代わりに、ランダムな小さなデータとして "バッチ" を使います。

この手法は [確率的学習](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) と言います。より安く、速く、そしてほぼ同じ結果を得ることができます。

In [ ]:
epochs_completed = 0
index_in_epoch = 0
num_examples = train_images.shape[0]

# バッチでのデータ投入
def next_batch(batch_size):
    
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # すべてのデータが学習に使われたらランダムに並び替える    
    if index_in_epoch > num_examples:
        # 終了エポック
        epochs_completed += 1
        # データのシャッフル
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        # 次のエポック
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]

さて、TensorFlow のグラフとして、すべての変数に対してすべての操作が定義できました。すべての計算は Python 外の環境で実行されます。

In [ ]:
# TensorFlow セッションの開始
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()

sess.run(init)

ループでの各ステップでは、学習データセットから "バッチ" を取得し、それを事前にグラフとして定義したプレースホルダと置換しながら学習を進めます。ここでは *x, y* や *dropout* にあたるものです。

また、時折、次回の "バッチ" に対して学習精度をチェックします。

ローカル環境では、[学習プロセスの保存](https://www.tensorflow.org/versions/master/api_docs/python/state_ops.html#Saver) をオススメします。これにより将来の学習やデバック、評価のためにデータをリカバリすることができるようになります。

In [ ]:
# 変数の可視化
train_accuracies = []
validation_accuracies = []
x_range = []

display_step=1

for i in range(TRAINING_ITERATIONS):

    # 新しいバッチの取得
    batch_xs, batch_ys = next_batch(BATCH_SIZE)        

    # 各 1, 2, ..., 10, 20, ..., 100... ステップでチェックする
    if i%display_step == 0 or (i+1) == TRAINING_ITERATIONS:
        
        train_accuracy = accuracy.eval(feed_dict={x:batch_xs, 
                                                  y_: batch_ys, 
                                                  keep_prob: 1.0})       
        if(VALIDATION_SIZE):
            validation_accuracy = accuracy.eval(feed_dict={ x: validation_images[0:BATCH_SIZE], 
                                                            y_: validation_labels[0:BATCH_SIZE], 
                                                            keep_prob: 1.0})                                  
            print('training_accuracy / validation_accuracy => %.2f / %.2f for step %d'%(train_accuracy, validation_accuracy, i))
            
            validation_accuracies.append(validation_accuracy)
            
        else:
             print('training_accuracy => %.4f for step %d'%(train_accuracy, i))
        train_accuracies.append(train_accuracy)
        x_range.append(i)
        
        # 表示用ステップを更新する
        if i%(display_step*10) == 0 and i:
            display_step *= 10

    # バッチを学習する
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: DROPOUT})

学習が終わったら、学習に使われなかったデータで精度を確認してみましょう。

In [ ]:
# 評価用セットで最終的な精度を確認する
if(VALIDATION_SIZE):
    validation_accuracy = accuracy.eval(feed_dict={x: validation_images, 
                                                   y_: validation_labels, 
                                                   keep_prob: 1.0})
    print('validation_accuracy => %.4f'%validation_accuracy)
    plt.plot(x_range, train_accuracies,'-b', label='Training')
    plt.plot(x_range, validation_accuracies,'-g', label='Validation')
    plt.legend(loc='lower right', frameon=False)
    plt.ylim(ymax = 1.1, ymin = 0.7)
    plt.ylabel('accuracy')
    plt.xlabel('step')
    plt.show()

結果が得られるのはうれしいことですが、*test.csv* からテストデータを読み込み、その画像群に対してもラベルを推論してみましょう。

テストデータは画像データのみを含んでおり、ラベル情報がありません。それ以外は、その構造は学習データと似ています。

推論されたラベルは CSV に保存されます。いずれ競技会へ Submit してみましょう。

In [ ]:
# CSV ファイルからテストデータを読み込む
test_images = pd.read_csv('./test.csv').values
test_images = test_images.astype(np.float)

# 値を変換する [0:255] => [0.0:1.0]
test_images = np.multiply(test_images, 1.0 / 255.0)
print('test_images({0[0]}, {0[1]})'.format(test_images.shape))

# テストデータでの推論
#predicted_lables = predict.eval(feed_dict={x: test_images, keep_prob: 1.0})

# リソースをより効率的に使うためにバッチで計算する
predicted_lables = np.zeros(test_images.shape[0])
for i in range(0,test_images.shape[0]//BATCH_SIZE):
    predicted_lables[i*BATCH_SIZE : (i+1)*BATCH_SIZE] = predict.eval(feed_dict={x: test_images[i*BATCH_SIZE : (i+1)*BATCH_SIZE], 
                                                                                keep_prob: 1.0})
print('predicted_lables({0})'.format(len(predicted_lables)))

# テスト画像とその推論結果を出力する
display(test_images[IMAGE_TO_DISPLAY])
print('predicted_lables[{0}] => {1}'.format(IMAGE_TO_DISPLAY,predicted_lables[IMAGE_TO_DISPLAY]))

# 結果を保存する
np.savetxt('submission_softmax.csv', 
           np.c_[range(1,len(test_images)+1),predicted_lables], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt='%d')

## 付録

以前示唆したように、処理をよりよく理解するためには変数を出力してみるのがよいでしょう。

以下は TensorFlow グラフから、最初の畳み込み層を出力するものです。32 の特徴を格子状に画像へ変換しますが、異なる画像からニューラルネットワークによって大まかな特徴がフィルタリングされる様子はとても興味深いかと思います。

In [ ]:
layer1_grid = layer1.eval(feed_dict={x: test_images[IMAGE_TO_DISPLAY:IMAGE_TO_DISPLAY+1], keep_prob: 1.0})
plt.axis('off')
plt.imshow(layer1_grid[0], cmap=cm.seismic )

In [ ]:
sess.close()

## 参考情報

- [Deep MNIST for Experts](https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#deep-mnist-for-experts)
- [A Convolutional Network implementation example using TensorFlow library](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3%20-%20Neural%20Networks/convolutional_network.ipynb)
- [Digit recognizer in Python using CNN](https://www.kaggle.com/kobakhit/digit-recognizer/digit-recognizer-in-python-using-cnn)
- [Deep Learning in a Nutshell: Core Concepts](http://devblogs.nvidia.com/parallelforall/deep-learning-nutshell-core-concepts/)